##### 부산시 구군별 인구데이터 불러오기

In [1]:
import pandas as pd

In [34]:
gugun_pop = pd.read_excel('./data/부산광역시_전체_세대.xlsx',header=1,usecols=[0,17,18,19,20,21,22,23,24])

c:\Users\dfgdd\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [58]:
gugun_pop.columns

Index(['구·군별', '읍면동수 (개).2', '세대수 (세대).2', '인구수  (명).2', '남자인구수 (명).2',
       '여자인구수 (명).2', '시전체 인구에 대한 구성비 (%).2', '면적 (㎢).2', '인구밀도 (명/㎢).2'],
      dtype='object')

In [60]:
## 필요 칼럼만 뽑아내기
gugun_pop = gugun_pop[['구·군별','면적 (㎢).2','인구수  (명).2']]


In [65]:
# 열 이름 변경하기
gugun_pop = gugun_pop.rename(columns={'구·군별': '구군명'})
gugun_pop = gugun_pop.rename(columns={'면적 (㎢).2': '면적'})
gugun_pop = gugun_pop.rename(columns={'인구수  (명).2':'인구수'})

In [66]:
gugun_pop.head()

,구군명,면적,인구수
0,부산광역시,771.33,3348011
1,중구,3.01,41103
2,서구,13.98,106500
3,동구,10.12,90028
4,영도구,14.20,108959


In [68]:
## 저장하기
gugun_pop.to_excel('./data/부산광역시_구군명_데이터_전처리.xlsx',index=False)

In [69]:
## 행정기관 갯수 데이터 읽어와서 합치기
admi = pd.read_excel('./data/구군별_행정기관갯수.xlsx')

In [71]:
df_data = pd.merge(left = gugun_pop,right=admi,how='left',on='구군명')

## 이상하지만 API 데이터에 동구 데이터는 없기에 삭제하고 진행

In [77]:
#count가 Nan값인 행들 삭제

df_data = df_data[df_data['count'].isnull() != True]

In [81]:
# 열 이름 변경하기
df_data = df_data.rename(columns={'count': '행정공공기관갯수'})

In [86]:
## 행정공공기관갯수 int타입으로 변환하기
df_data['행정공공기관갯수'] = df_data['행정공공기관갯수'].astype('Int64')

In [87]:
df_data.head()

,구군명,면적,인구수,행정공공기관갯수
1,중구,3.01,41103,18
2,서구,13.98,106500,15
4,영도구,14.20,108959,18
5,부산진구,29.67,363092,21
6,동래구,16.63,271164,14
